In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from neural_network import baseline_model, residual_model

In [ ]:
train_data_dir = '/path/to/train/dataset'
validation_data_dir = '/path/to/validation/dataset'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(101, 101),
    batch_size=32,
    class_mode='binary')

validation_generator = valid_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(101, 101),
    batch_size=32,
    class_mode='binary')


In [ ]:
# Function to identify input size of the dataset
def identify_dataset_size(generator):
    # Get the number of classes
    num_classes = generator.num_classes

    # Get the total number of images and the size of each image
    total_images = len(generator.filenames)
    image_shape = generator.image_shape

    # Get the number of images per class
    class_counts = {}
    for class_label in generator.class_indices:
        class_name = os.path.basename(class_label)
        class_count = sum(1 for img in generator.filenames if img.startswith(class_label))
        class_counts[class_name] = class_count

    return num_classes, total_images, image_shape, class_counts

# Identify input size of the dataset
num_classes_train, total_images_train, image_shape_train, class_counts_train = identify_dataset_size(train_generator)
num_classes_valid, total_images_valid, image_shape_valid, class_counts_valid = identify_dataset_size(validation_generator)

print("Training Dataset:")
print(f"Number of classes: {num_classes_train}")
print(f"Total number of images: {total_images_train}")
print(f"Image shape: {image_shape_train}")
print("Number of images per class:")
for class_name, count in class_counts_train.items():
    print(f"{class_name}: {count}")

print("\nValidation Dataset:")
print(f"Number of classes: {num_classes_valid}")
print(f"Total number of images: {total_images_valid}")
print(f"Image shape: {image_shape_valid}")
print("Number of images per class:")
for class_name, count in class_counts_valid.items():
    print(f"{class_name}: {count}")


In [ ]:
baseline_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
baseline_history = baseline_model.fit(train_generator, validation_data=validation_generator, epochs=10)


In [ ]:
residual_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
residual_history = residual_model.fit(train_generator, validation_data=validation_generator, epochs=10)

In [ ]:
# Calculate ROC and AUC for baseline model
baseline_y_pred = baseline_model.predict(validation_generator)
baseline_fpr, baseline_tpr, _ = roc_curve(validation_generator.classes, baseline_y_pred)
baseline_auc = roc_auc_score(validation_generator.classes, baseline_y_pred)


In [ ]:
# Calculate ROC and AUC for residual model
residual_y_pred = residual_model.predict(validation_generator)
residual_fpr, residual_tpr, _ = roc_curve(validation_generator.classes, residual_y_pred)
residual_auc = roc_auc_score(validation_generator.classes, residual_y_pred)

In [ ]:
# Get predictions and ground truth labels for classification report
baseline_y_pred_classes = np.argmax(baseline_y_pred, axis=1)

In [ ]:
residual_y_pred_classes = np.argmax(residual_y_pred, axis=1)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(baseline_fpr, baseline_tpr, label=f'Baseline Model (AUC = {baseline_auc:.2f})')
plt.plot(residual_fpr, residual_tpr, label=f'Residual Model (AUC = {residual_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()


In [ ]:
print("Baseline Model Classification Report:")
print(classification_report(validation_generator.classes, baseline_y_pred_classes))


In [ ]:
print("Residual Model Classification Report:")
print(classification_report(validation_generator.classes, residual_y_pred_classes))